# Zonal statistics

In [ ]:
# Load raster
import rioxarray
burned = rioxarray.open_rasterio('burned.tif')
burned

In [ ]:
# Load assets vector
import geopandas as gpd
assets = gpd.read_file('assets.gpkg')
assets

## Align the CRS of the two datasets

In [ ]:
burned.rio.crs

In [ ]:
assets.crs

In [ ]:
assets = assets.to_crs(burned.rio.crs)

# Rasterizing the vector data

In [ ]:
geom = assets[['geometry', 'code']].values.tolist()
geom

In [ ]:
# The image has 1 band dimension
burned.shape

In [ ]:
# A squeeze is needed to make the raster 2D
# Since rasterize function only accept 2D shape
burned.squeeze().shape

In [ ]:
burned_squeeze = burned.squeeze()

In [ ]:
# Rasterize the vector data.
# transform represents the projection from pixel space to the projected coordinate space. 
# By default, the pixels that are not contained within a polygon in our shapefile will be filled with 0.
from rasterio import features
assets_rasterized = features.rasterize(geom, out_shape=burned_squeeze.shape, transform=burned.rio.transform())

In [ ]:
assets_rasterized

## Create an Xarray.DataArray from the rasterized results

In [ ]:
import numpy as np
print(assets_rasterized.shape)
print(np.unique(assets_rasterized))

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(assets_rasterized)
plt.colorbar()

In [ ]:
assets_rasterized_xarr = burned_squeeze.copy()
assets_rasterized_xarr.data = assets_rasterized

# visualize zones
assets_rasterized_xarr.plot()

## Perform Zonal Statistcs

In [ ]:
from xrspatial import zonal_stats
stats = zonal_stats(assets_rasterized_xarr, burned_squeeze)

In [ ]:
stats

Exercise: Zonal stats over slope zones

Now let's look into the burned areas in relation to slope classes like [Zhai et al., 2023](https://www.mdpi.com/1999-4907/14/4/807) have done. To reproduce their analysis, we will:
1. Reclassify the slope classes into 5 categories  0%–5%, 5%–10%, 10%–15%, 15%–25%, and above 25%, then,
2. Perform the zonal statistics on the above categories.

Hint:
1. Load slope data from 'slope_dask.tif'.
2. The big chellenge will be how to compute the slope zones. Consider:
    - Convert slope data to zones using [`numpy.digitize`](https://numpy.org/doc/stable/reference/generated/numpy.digitize.html).
    - Use [`xarray.apply_ufunc`](https://docs.xarray.dev/en/stable/generated/xarray.apply_ufunc.html) to apply `numpy.digitize` to all the elements in an Xarray. 
4. Note that the slope data and burn index data are in different resolution.

In [ ]:
import numpy as np
import xarray as xr

# Load data and remove redundant dimension
# slope = rioxarray.open_rasterio('slope.tif').squeeze()
slope = rioxarray.open_rasterio('slope_dask.tif', masked=True).squeeze()

# Defines the bins for pixel values
# Zone 0 will be values <=0.; Zone 5 will be values >0.25; Zone 6 will be NaN values(>1000)
slope_bins = (0., 0.05, 0.1, 0.15, 0.25, 1000)

slope_zones = xr.apply_ufunc(
    np.digitize,
    slope,
    slope_bins
)

# Reproject burned to slope
# Because slope zones has lower resolution
burned = rioxarray.open_rasterio('burned.tif').squeeze()
burned_match = burned.rio.reproject_match(slope_zones)

# Compute Zonal stats
stats = zonal_stats(slope_zones, burned_match)


In [ ]:
# Visualize slope zones
slope_zones.plot()

In [ ]:
stats